In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
# show all outputs of a cell (such as if df.head() and df.tail() are in the same cell)
#default is 'last_expr'

%matplotlib inline
import pandas as pd
import sklearn
import pickle
import numpy as np
from pandasql import sqldf
import matplotlib.pyplot as plt

sql = lambda q: sqldf(q, globals()) 

product=[('B','1121','PC'),('A','1232','PC'),('A','1233','PC'),('E','1260','PC'),('A','1276','Printer'),\
   ('D','1288','Printer'),('A','1298','Laptop'),('C','1321','Laptop'),('A','1401','Printer'),('A','1408','Printer'),\
   ('D','1433','Printer'),('E','1434','Printer'),('B','1750','Laptop'),('A','1752','Laptop'),('E','2113','PC'),\
   ('E','2112','PC')]
product=pd.DataFrame(product,columns=['maker','model','type'])

pc=[(1,'1232',500,64,5,'12x',600),(2,'1121',750,128,14,'40x',850),(3,'1233',500,64,5,'12x',600),\
(4,'1121',600,128,14,'40x',850),(5,'1121',600,128,8,'40x',850),(6,'1233',750,128,20,'50x',950),\
(7,'1232',500,32,10,'12x',400),(8,'1232',450,64,8,'24x',350),(9,'1232',450,32,10,'24x',350),\
(10,'1260',500,32,10,'12x',350),(11,'1233',900,128,40,'40x',980),(12,'1233',800,128,20,'50x',970)]
pc=pd.DataFrame(pc,columns=['code','model','speed','ram','hd','cd','price'])           

laptop=[(1,'1298',350,32,4,700,11),(2,'1321',500,64,8,970,12),(3,'1750',750,128,12,1200,14),\
(4,'1298',600,64,10,1050,15),(5,'1752',750,128,10,1150,14),(6,'1298',450,64,10,950,12)]
laptop=pd.DataFrame(laptop,columns=['code','model','speed','ram','hd','price','screen'])

printer=[(1,'1276','n','Laser',400),(2,'1433','y','Jet',270),(3,'1434','y','Jet',290),\
(4,'1401','n','Matrix',150),(5,'1408','n','Matrix',270),(6,'1288','n','Laser',400)]
printer=pd.DataFrame(printer,columns=['code','model','color','type','price'])

**Exercise: 58 (Serge I: 2009-11-13) <br>
For each product type and maker in the Product table, find out, with a precision of two decimal places, the percentage ratio of the number of models of the actual type produced by the actual maker to the total number of models by this maker.
Result set: maker, product type, the percentage ratio mentioned above.**

In [ ]:
#product type, maker:  (# of models for combo / # of models by same maker)

for each maker,product combo count the number of rows

In [13]:
q='''
select maker,type, 
(select count(*) from product p1 where p1.maker=p3.maker and p1.type=p3.type)/
(select count(*) from product p2 where p2.maker=p3.maker) as percentage
from product p3
group by maker, type


'''
sql(q)

,maker,type,percentage
0,A,Laptop,0
1,A,PC,0
2,A,Printer,0
3,B,Laptop,0
4,B,PC,0
5,C,Laptop,1
6,D,Printer,1
7,E,PC,0
8,E,Printer,0


**Exercise: 47 (Serge I: 2011-02-11) <br>
Number the rows of the Product table as follows: makers in descending order of number of models produced by them (for manufacturers producing an equal number of models, their names are sorted in ascending alphabetical order); model numbers in ascending order.<br>
Result set: row number as described above, manufacturer's name (maker), model. **

In [4]:
Product=product

create table with maker, model, and total models per maker (which is the same regardless of model number)
join this table with itself
for each record in first table, you'll have columns from second table that have a count greater than maker count of first table

In [51]:
q='''
select t1.maker, t1.model,t1.c,t2.maker,t2.model,t2.c
from (select maker, model, c from Product
      join (select count(model) c, maker m from Product group by maker ) b1
      on maker = m) t1
join (select maker, model, c from Product 
      join (select count(model) c, maker m from Product group by maker ) b2
on maker = m) t2

on t2.c > t1.c 
or t2.c=t1.c and t2.maker<t1.maker
or t2.c=t1.c and t2.maker=t1.maker and t2.model <= t1.model
order by 1


'''
sql(q);

for each maker, model pairing
     the number of records for this pairing is increased by the maker, model records associated with makers that have more models.  
     
If the maker with the most models, then first maker, model has 1 record.
Second maker, model (model # above first records'),   

maker with largest count of records is first
this maker, for each of its models, has a record for all models of makers with more models, for all models of makers with the same number of models where the maker's name is a lower value alphabetically (makers with higher values get more records), or for the same maker
(so this maker, if it originally had n models, will then have nxk models where k is the 
number of models for all makers with more models than this maker or the same number,)


In [52]:
q='''
select count(*) num, t1.maker, t1.model
from (select maker, model, c from Product
      join (select count(model) c, maker m from Product group by maker ) b1
      on maker = m) t1
join (select maker, model, c from Product 
      join (select count(model) c, maker m from Product group by maker ) b2
on maker = m) t2

on t2.c > t1.c 
or t2.c=t1.c and t2.maker<t1.maker
or t2.c=t1.c and t2.maker=t1.maker and t2.model <= t1.model
group by t1.maker, t1.model
order by 1;


'''
sql(q)

,num,t1.maker,t1.model
0,1,A,1232
1,2,A,1233
2,3,A,1276
3,4,A,1298
4,5,A,1401
5,6,A,1408
6,7,A,1752
7,8,E,1260
8,9,E,1434
9,10,E,2112


**Exercise: 41 (Serge I: 2008-08-30) <br>
For the PC in the PC table with the maximum code value, obtain all its characteristics (except for the code) and display them in two columns 
**
- name of the characteristic (title of the corresponding column in the PC table);
- its respective value. 

In [8]:
q='''
select 'model', cast(model as NVARCHAR (10)) as model from pc where code in (select max(code) from pc)
union
select 'speed', cast(speed as NVARCHAR (10)) from pc where code in (select max(code) from pc)
union
select 'ram', cast(ram as NVARCHAR (10)) from pc where code in (select max(code) from pc)
union
select 'hd', cast(hd as NVARCHAR (10)) from pc where code in (select max(code) from pc)
union
select 'cd', cast(cd as NVARCHAR (10)) from pc where code in (select max(code) from pc)
union
select 'price', cast(price as NVARCHAR (10)) from pc where code in (select max(code) from pc)

'''
sql(q)

,'model',model
0,cd,50x
1,hd,20
2,model,1233
3,price,970
4,ram,128
5,speed,800


**Exercise: 35 (qwrqwr: 2012-11-23) <br>
Find models in the Product table consisting either of digits only or Latin letters (A-Z, case insensitive) only.<br>
Result set: model, type.**

In [23]:
#sqlite uses % and _ for the like operator, but does NOT accept character lists []
#[^charlist] or [%charlist] any character other than the charlist
#therefore, none of the below can be done in sqlite

#like '%[0-9]%' returns strings with a number ANYWHERE in them (with at least one number)
#like '%[^0-9]%' returns strings that are not ALL numbers
    #the string'hi9' would match since if we just examine the '%[^0-9]' part, this means all 
        #strings with letters before 0-9 are returned since % is a wildcard '__9' 
    #the string '9hi' would match since if we just examine the '[^0-9]%' part, this means all
        # strings with letters after 0-9 are returned since % is a wildcard '9__'
    #the string 'hi' would match since it matches '[^0-9]%' and '%[^0-9]'  
#not like '%[^0-9]%' returns strings that are ALL numbers 
    #(=not strings that are not ALL #s means strings that ARE all numbers)

q='''
select model, type from product 
where model not like '%[^a-z]%' or model not like '%[^0-9]%'
'''
sql(q)

,model,type
0,1121,PC
1,1232,PC
2,1233,PC
3,1260,PC
4,1276,Printer
5,1288,Printer
6,1298,Laptop
7,1321,Laptop
8,1401,Printer
9,1408,Printer


**Exercise: 27 (Serge I: 2003-02-03) <br>
Find out the average hard disk drive capacity of PCs produced by makers who also manufacture printers.<br>
Result set: maker, average HDD capacity.**

In [2]:
q='''
select maker, avg(pc.hd) as Avg_hd from product p
left join pc on pc.model=p.model
where maker in (select maker from product where type='Printer')
group by maker
having count(pc.model)>0
'''
sql(q)

,maker,Avg_hd
0,A,14.75
1,E,10.00


**Exercise: 26 (Serge I: 2003-02-14)<br>
Find out the average price of PCs and laptops produced by maker A. <br>
Result set: one overall average price for all items.** 


In [10]:
#use union all, which does not remove duplicates
#You get the wrong answer with union, which removes duplicates, which in this case some
#price values of laptop are also in pc
q='''
with temp as (
select price from pc join product p on p.model=pc.model where p.maker='A'
union all
select price from laptop l join product p on p.model=l.model where p.maker='A'
)
select avg(price) from temp
'''
sql(q)

,avg(price)
0,754.166667


**Exercise: 25 <br>
Find the printer makers also producing PCs with the lowest RAM capacity and the highest processor speed of all PCs having the lowest RAM capacity. <br>
Result set: maker. **

In [49]:
#Included where conditions on the joined table
q='''
select distinct maker from product p
join pc on pc.model=p.model and pc.ram in (select min(ram) from pc)
and pc.speed >= (select max(speed) from pc where pc.ram in (select min(ram) from pc) )
where maker in (select maker from product where type = 'Printer')

'''
sql(q)

,maker
0,A
1,E


## Trying out different things

In [160]:
q='''
select 
sum(price) over(order by speed ascending between 2 preceding and current row) as running_avg
from pc
'''
sql(q)

PandaSQLException: (sqlite3.OperationalError) near "(": syntax error [SQL: '\nselect \nsum(price) over(order by speed ascending) as running_avg\nfrom pc\n']

In [153]:
q='''
select ram,
case when price >= 850 then "high" 
            when price >=600  then "medium"
            else "low"
            end as price_level, 
            count(1) as price_level_count
from pc
group by ram, case when price >= 850 then "high" 
            when price >=600  then "medium"
            else "low"
            end

'''
sql(q)

,ram,price_level,price_level_count
0,32,low,3
1,64,low,1
2,64,medium,2
3,128,high,6


In [150]:
q='''
select *,
case when price >= 850 then "high" 
            when price >=600  then "medium"
            else "low"
            end as price_level
from pc

'''
sql(q)

,code,model,speed,ram,hd,cd,price
0,1,1232,500,64,5,12x,600
1,2,1121,750,128,14,40x,850
2,3,1233,500,64,5,12x,600
3,4,1121,600,128,14,40x,850
4,5,1121,600,128,8,40x,850
5,6,1233,750,128,20,50x,950
6,7,1232,500,32,10,12x,400
7,8,1232,450,64,8,24x,350
8,9,1232,450,32,10,24x,350
9,10,1260,500,32,10,12x,350


In [123]:
q='''
select * from pc
LIMIT 5
'''
sql(q)

,code,model,speed,ram,hd,cd,price
0,1,1232,500,64,5,12x,600
1,2,1121,750,128,14,40x,850
2,3,1233,500,64,5,12x,600
3,4,1121,600,128,14,40x,850
4,5,1121,600,128,8,40x,850


In [121]:
q='''
with temp as (select ram, avg(price) as avg_price from pc group by ram)
select ram, max(avg_price) from temp
'''
sql(q)

,ram,max(avg_price)
0,128,908.333333


In [120]:
q='''
select ram, max(avg_price) as highest_avg 
from (select avg(price) as avg_price, ram from pc group by ram)
'''
sql(q)

,ram,highest_avg
0,128,908.333333


In [58]:
q='''
select * from pc
'''
sql(q)

,code,model,speed,ram,hd,cd,price
0,1,1232,500,64,5,12x,600
1,2,1121,750,128,14,40x,850
2,3,1233,500,64,5,12x,600
3,4,1121,600,128,14,40x,850
4,5,1121,600,128,8,40x,850
5,6,1233,750,128,20,50x,950
6,7,1232,500,32,10,12x,400
7,8,1232,450,64,8,24x,350
8,9,1232,450,32,10,24x,350
9,10,1260,500,32,10,12x,350


In [48]:
q='''
select distinct maker, pc.model
from product, pc
where product.model = pc.model
'''
sql(q)

,maker,model
0,B,1121
1,A,1232
2,A,1233
3,E,1260


In [24]:
q='''
select * from product
left outer join pc 
on pc.model = product.model

'''
sql(q)

,maker,model,type,code,model,speed,ram,hd,cd,price
0,B,1121,PC,2.0,1121,750.0,128.0,14.0,40x,850.0
1,B,1121,PC,4.0,1121,600.0,128.0,14.0,40x,850.0
2,B,1121,PC,5.0,1121,600.0,128.0,8.0,40x,850.0
3,A,1232,PC,1.0,1232,500.0,64.0,5.0,12x,600.0
4,A,1232,PC,7.0,1232,500.0,32.0,10.0,12x,400.0
5,A,1232,PC,8.0,1232,450.0,64.0,8.0,24x,350.0
6,A,1232,PC,9.0,1232,450.0,32.0,10.0,24x,350.0
7,A,1233,PC,3.0,1233,500.0,64.0,5.0,12x,600.0
8,A,1233,PC,6.0,1233,750.0,128.0,20.0,50x,950.0
9,A,1233,PC,11.0,1233,900.0,128.0,40.0,40x,980.0


In [18]:
q='''
select * from product p
join pc on pc.model = p.model
    and pc.speed > 600 
    and p.maker = 'A'
'''
sql(q)

q='''
select * from product p
where p.maker = 'A'
join pc on pc.model = p.model
    and pc.speed > 600 

'''
sql(q)

q='''
select * from product p
join pc on pc.model = p.model
    and pc.speed > 600 
where p.maker = 'A'
'''
sql(q)

,maker,model,type,code,model,speed,ram,hd,cd,price
0,A,1233,PC,6,1233,750,128,20,50x,950
1,A,1233,PC,11,1233,900,128,40,40x,980
2,A,1233,PC,12,1233,800,128,20,50x,970


PandaSQLException: (sqlite3.OperationalError) near "join": syntax error [SQL: "\nselect * from product p\nwhere p.maker = 'A'\njoin pc on pc.model = p.model\nand pc.speed > 600 \n\n"]

**Exercise: 14 (Serge I: 2012-04-20)<br>
Get the makers who produce only one product type and more than one model. Output: maker, type.**

In [2]:
q='''
select distinct maker, type from product
where maker in 
(select maker from product group by maker
having count(distinct type) =1 and 
count(distinct model)>1)
'''
sql(q)

,maker,type
0,D,Printer


In [11]:
q='''
select maker, type from 
(select maker, type from product group by maker
having count(distinct type)=1 and count(distinct model)>1) 
'''
sql(q)

,maker,type
0,D,Printer


**Exercise: 15 (Serge I: 2003-02-03)<br>
Get hard drive capacities that are identical for two or more PCs. <br>
Result set: hd.**

In [17]:
q='''
select hd from pc
group by hd
having count(hd)>=2
'''
sql(q)

,hd
0,5.0
1,8.0
2,10.0
3,14.0
4,20.0


**Exercise: 16 (Serge I: 2003-02-03)
Get pairs of PC models with identical speeds and the same RAM capacity.<br>
Each resulting pair should be displayed only once, i.e. (i, j) but not (j, i). <br>
Result set: model with the bigger number, model with the smaller number, speed, and RAM.**

In [26]:
q=''' 
select a.model, b.model, a.speed, a.ram from pc as a, pc as b
where a.speed=b.speed and a.ram=b.ram and a.model>b.model
'''
sql(q) #notice two tables are in the from statement

,model,model,speed,ram
0,1233,1232,500.0,64.0
1,1233,1121,750.0,128.0
2,1260,1232,500.0,32.0


**Exercise: 17 (Serge I: 2003-02-03) <br>
Get the laptop models that have a speed smaller than the speed of any PC.  <br>
Result set: type, model, speed.**

In [59]:
q=''' 
select distinct 'laptop' as type, model, speed
from laptop
where speed < (select min(pc.speed) from pc)
''' # notice that pc is not needed in the from statement
    # notice that we don't need to refer to laptop since the subquery in the where
    # clause is not conflicting with the laptop table
sql(q)

,type,model,speed
0,laptop,1298,350.0


**Find the makers of the cheapest color printers.<br>
Result set: maker, price.**

In [3]:
q='''
select p2.maker, p1.price
from printer p1 
join product p2 on p1.model = p2.model
where color='y' and p1.price in (select min(price) from printer where color='y')
order by p1.price asc
''' #You can use aggregate functions without a group by!  
sql(q)

,maker,price
0,D,270.0


**Exercise: 19 (Serge I: 2003-02-13)<br>
For each maker having models in the Laptop table, find out the average screen size of the laptops he produces. <br>
Result set: maker, average screen size.<br>**


In [4]:
q='''
select p.maker, avg(l.screen)
from laptop l
join product p on p.model = l.model
group by maker
'''
sql(q)

,maker,avg(l.screen)
0,A,13.0
1,B,14.0
2,C,12.0


**Exercise: 20 (Serge I: 2003-02-13) <br>
Find the makers producing at least three distinct models of PCs. <br>
Result set: maker, number of PC models. <br>**

In [15]:
q='''
select maker, count(distinct model)
from products
where type='PC'
group by maker
having count(distinct model) >=3
'''
sql(q)

,maker,count(distinct model)
0,E,3


**Exercise: 21 (Serge I: 2003-02-13)<br>
Find out the maximum PC price for each maker having models in the PC table. Result set: maker, maximum price.<br>**


In [3]:
q='''
select p.maker, max(pc.price) as maximum_price
from product p join pc on p.model=pc.model
group by maker
'''
sql(q)

,maker,maximum_price
0,A,980.0
1,B,850.0
2,E,350.0


**Exercise: 22 (Serge I: 2003-02-13) <br>
For each value of PC speed that exceeds 600 MHz, find out the average price of PCs with identical speeds. <br>
Result set: speed, average price. <br>**


In [4]:
q='''
select speed, avg(price)
from pc
where speed>600
group by speed
'''
sql(q)

,speed,avg(price)
0,750.0,900.0
1,800.0,970.0
2,900.0,980.0


**Exercise: 23 (Serge I: 2003-02-14) <br>
Get the makers producing both PCs having a speed of 750 MHz or higher and laptops with a speed of 750 MHz or higher.  In other words, makers with at least one pc whose speed is 750 or higher AND at least one laptop whose speed is 750 or higher. <br>
Result set: maker**

In [22]:

q='''
select * from laptop
'''
sql(q)

,code,model,speed,ram,hd,price,screen
0,1,1298,350,32,4,700,11
1,2,1321,500,64,8,970,12
2,3,1750,750,128,12,1200,14
3,4,1298,600,64,10,1050,15
4,5,1752,750,128,10,1150,14
5,6,1298,450,64,10,950,12


In [27]:
# makers (where pc speed>=750) and (laptop speed >=750)
q='''
select maker from product
where model in (select model from laptop where speed>=750)
and model in (select model from pc where speed >=750)
'''
sql(q)

,maker


In [37]:
#using group by and having clause
q='''
select p.maker from product p 
group by maker having maker in
(select maker from product p join laptop l on l.model=p.model where l.speed>=750) and maker in 
(select maker from product p join pc on pc.model=p.model where pc.speed>=750)
'''
sql(q)

,maker
0,A
1,B


In [43]:
#using where and distinct
q='''
select distinct maker from product where maker in 
(select maker from product p join laptop l on l.model=p.model where l.speed =750) and maker in 
(select maker from product p join pc on pc.model=p.model where pc.speed =750)
'''
sql(q)

,maker
0,A
1,B


In [45]:
#using multiple subqueries
q='''
select distinct maker from product where maker in 
(select maker from product where model in (select model from laptop where laptop.speed>=750))
and maker in 
(select maker from product where model in (select model from pc where pc.speed =750))
'''
sql(q)

,maker
0,A
1,B


In [47]:
#using aggregate functions in having clause for joined tables
q='''
select maker from product p 
left join laptop l on p.model=l.model 
left join pc on pc.model=p.model
group by maker
having max(pc.speed)>=750 and max(l.speed)>=750
'''
sql(q)

,maker
0,A
1,B


**Exercise: 24 (Serge I: 2003-02-03)<br>
List the models of any type having the highest price of all products present in the database.**

In [15]:
#use the with clause to create a common table expression (CTE) that is like a derived table in that it lasts for 
#the duration of the query, EXCEPT it can be referenced multiple times in the query such as with WHERE and group by
#it essentially acts as a temporary result-set 
#the where clause does not use where price >= ALL(select price from sub) because sqlite can't use ALL in this way
q='''
with sub as (
select model, price from pc where price in (select max(price) from pc) union
select model, price from laptop where price in (select max(price) from laptop) union
select model, price from printer where price in (select max(price) from printer)  )
select model from sub
where price = (select max(price) from sub)

'''
sql(q)

,model
0,1750


In [38]:
q='''
select * from laptop limit 1
'''
sql(q)

,code,model,speed,ram,hd,price,screen
0,1,1298,350,32,4,700,11


** Exercise: 10 (Serge I: 2002-09-23) <br>
Find the printer models having the highest price. Result set: model, price.**

In [7]:
q='''
select model, max(price) from printer 
'''
sql(q)

,model,max(price)
0,1276,400


** Exercise: 9 (Serge I: 2002-11-02) <br>
Find the makers of PCs with a processor speed of 450 MHz or more. Result set: maker. **

In [120]:
q='''
select maker from product 
join pc on product.model = pc.model 
where speed >= 450
group by maker 

'''
sql(q)

,maker
0,A
1,B
2,E


In [117]:
q='''
select maker from product 
join pc on product.model = pc.model and pc.speed >= 450
group by maker 

'''
sql(q)

,maker
0,A
1,B
2,E


** Exercise: 8 (Serge I: 2003-02-03)
Find the makers producing PCs but not laptops. ** 

In [106]:
#return makers NOT producing laptops, return only of these that are producing PCs
q='''
select maker from product where type = 'PC' and maker not in 
(select maker from product where type = 'Laptop' group by Maker)
group by maker

'''
sql(q)

,maker
0,E


**Exercise: 7 (Serge I: 2002-11-02) <br>
Get the models and prices for all commercially available products (of any type) produced by maker B. **

In [77]:
q='''
select pc.model, pc.price from pc
join product p on p.model = pc.model
where p.maker = "B"

union

select l.model, l.price from laptop l
join product p on p.model = l.model
where p.maker = "B"

union 

select pr.model, pr.price from printer pr
join product p on p.model = pr.model
where p.maker = "B"
'''
sql(q)

,model,price
0,1121,850
1,1750,1200


In [71]:
q='''
with temp as (
select model, price from pc
union
select model, price from laptop
union
select model, price from printer
)
select temp.model,temp.price from temp
join product on product.model = temp.model
where maker="B"
'''
sql(q)

,model,price
0,1121,850
1,1750,1200


**Exercise: 6 (Serge I: 2002-10-28) <br>
For each maker producing laptops with a hard drive capacity of 10 Gb or higher, find the speed of such laptops. Result set: maker, speed.**

In [48]:
q='''
select p.maker, l.speed from laptop l 
join product p on p.model = l.model
where hd>=10
group by p.maker, l.speed
'''
sql(q)

,maker,speed
0,A,450
1,A,600
2,A,750
3,B,750


**Exercise: 5 (Serge I: 2002-09-30) <br>
Find the model number, speed and hard drive capacity of PCs cheaper than $600 having a 12x or a 24x CD drive.**


In [36]:
q='''
select model, speed, hd from pc 
where price <600 and cd in ('12x','24x')
'''
sql(q)

,model,speed,hd
0,1232,500,10
1,1232,450,8
2,1232,450,10
3,1260,500,10


**Exercise: 4 (Serge I: 2002-09-21) <br>
Find all records from the Printer table containing data about color printers.**


In [34]:
q='''
select * from printer where color = 'y'
'''
sql(q)

,code,model,color,type,price
0,2,1433,y,Jet,270
1,3,1434,y,Jet,290


** Exercise: 3 (Serge I: 2002-09-30) <br>
Find the model number, RAM and screen size of the laptops with prices over $1000. **

In [27]:
q='''
select model, ram, screen from laptop where price>1000
'''
sql(q)

,model,ram,screen
0,1750,128,14
1,1298,64,15
2,1752,128,14


**Exercise: 2 (Serge I: 2002-09-21) <br>
List all printer makers. Result set: maker.**


In [25]:
q='''
select maker from product where type = 'Printer' group by maker
'''
sql(q)

,maker
0,A
1,D
2,E


In [19]:
q='''
select maker from product p1
join printer p2 on p1.model = p2.model
group by maker
'''
sql(q)

,maker
0,A
1,D
2,E


** Exercise: 1 (Serge I: 2002-09-30) <br> 
Find the model number, speed and hard drive capacity for all the PCs with prices below $500.
Result set: model, speed, hd.**

In [4]:
q='''
select model, speed, hd from pc
where price <500
'''
sql(q)

,model,speed,hd
0,1232,500,10
1,1232,450,8
2,1232,450,10
3,1260,500,10
